### Importing the libraries

In [1]:
#pip install tensorflow==2.0.0

In [1]:
import tensorflow as tf

The version is: 2.0.0


In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Flatten,Dense,Dropout

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# path to your dataset
path = 'Replace the path of your dataset here'

In [5]:
folders = os.listdir(path)
folders

['without_mask', 'with_mask']

### Loading the images and labeling them

In [6]:
#initializations
data = []
#looping through all the folders
for folder in folders: 
    new_path = os.path.join(path,folder)
    total = len(os.listdir(new_path))
    label = folders.index(folder)
    print('folder_name = {} and no of images = {}'.format(folder,total))
    for img in os.listdir(new_path):#loop through all the images in that folder
    #read the images one by one
      try:
          img = cv2.imread( os.path.join(new_path,img),0) #grayscale
          img = cv2.resize(img,(100,100)) #check here once
          data.append([img,label])
      except Exception as e:
          print('Exception ',e)

folder_name = without_mask and no of images = 686
folder_name = with_mask and no of images = 690


### Shuffling the images randomly
* As we have loaded the images sequentially, it is better to shuffle the images randomly.

In [17]:
#shuffle the data in place
import random
random.shuffle(data)

In [18]:
#seperating images are labels after shuffling
images = []
labels = []
for image,label in data:
    images.append(image)
    labels.append(label)

In [19]:
#normalizing the images 
images = np.array(images)/255
labels = np.array(labels)

In [20]:
images.shape

(1376, 100, 100)

In [21]:
#converting to 4D 
images = np.expand_dims(images,axis = 3)

In [22]:
images.shape

(1376, 100, 100, 1)

In [23]:
# seperating train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(images,labels,test_size = 0.1,random_state = 0)

### Model Building

In [24]:
#initializing the model
model=Sequential()

#The first CNN layer followed by Relu and MaxPooling layers
model.add(Conv2D(200,(3,3),activation = 'relu',input_shape=images.shape[1:]))
model.add(MaxPooling2D())

#The second convolution layer followed by Relu and MaxPooling layers
model.add(Conv2D(100,(3,3),activation = 'relu'))
model.add(MaxPooling2D())

#Flattening 
model.add(Flatten())

#Adding dropout to avoid overfitting
model.add(Dropout(0.5))

#Dense Layer of 50 neurons
model.add(Dense(50,activation='relu'))

#Output layer
model.add(Dense(1,activation='sigmoid'))

In [25]:
#summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 52900)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 52900)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)               

In [26]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

### Model Training and Evaluation

In [27]:
# BATCH_SIZE = 32
EPOCHS = 10
history=model.fit(X_train,y_train,epochs=EPOCHS, validation_data = (X_test,y_test)) #,callbacks=[checkpoint]

Train on 1238 samples, validate on 138 samples
Epoch 1/10
1238/1238 [==============================] - 89s 72ms/sample - loss: 0.6845 - accuracy: 0.6042 - val_loss: 0.5769 - val_accuracy: 0.7536
Epoch 2/10
1238/1238 [==============================] - 85s 69ms/sample - loss: 0.4791 - accuracy: 0.7722 - val_loss: 0.4115 - val_accuracy: 0.8623
Epoch 3/10
1238/1238 [==============================] - 86s 70ms/sample - loss: 0.3303 - accuracy: 0.8643 - val_loss: 0.2632 - val_accuracy: 0.8841
Epoch 4/10
1238/1238 [==============================] - 83s 67ms/sample - loss: 0.2170 - accuracy: 0.9208 - val_loss: 0.2066 - val_accuracy: 0.9275
Epoch 5/10
1238/1238 [==============================] - 81s 65ms/sample - loss: 0.1700 - accuracy: 0.9394 - val_loss: 0.1722 - val_accuracy: 0.9348
Epoch 6/10
1238/1238 [==============================] - 82s 66ms/sample - loss: 0.1366 - accuracy: 0.9540 - val_loss: 0.1752 - val_accuracy: 0.9348
Epoch 7/10
1238/1238 [==============================] - 82s 66ms/

### Save the model

In [28]:
mask = 'model.h5'
path = 'Replace the path with where you want to save your model' + '/' + mask


In [29]:
#save the model
model.save('/content/drive/My Drive/FACE MASK/mask.h5')